Оригинальный код: https://www.kaggle.com/anokas/simple-xgboost-starter-0-0655

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import gc

In [4]:
print('Загрузка данных ...')

train = pd.read_csv('drive/MyDrive/zillow-prize-1/train_2016_v2.csv')
prop = pd.read_csv('drive/MyDrive/zillow-prize-1/properties_2016.csv')
sample = pd.read_csv('drive/MyDrive/zillow-prize-1/sample_submission.csv')

Загрузка данных ...


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Приведение к типу float32
print('Binding to float32')

for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)

Binding to float32


In [6]:
print('Создание обучающего набора данных ...')

#Объединение таблиц train и properties
df_train = train.merge(prop, how='left', on='parcelid')

#Удаление столбцов
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)

#Записать значения logerror в y_train
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

#Записать столбцы из x_train
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

#Поделить данные по 80000 значению
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

Создание обучающего набора данных ...
(90275, 55) (90275,)


In [7]:
print('Построение DMatrix...')

#Перевод в формат DMatrix
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

del x_train, x_valid; gc.collect()

Построение DMatrix...


278

In [8]:
print('Обучение ...')

#Задание парметров модели
params = {}
params['eta'] = 0.02
  #Использование линейной регрессии
params['objective'] = 'reg:linear'
  #mae Средеяя абсолютная ошибка
params['eval_metric'] = 'mae'
  #Максимальная глубина дерева
params['max_depth'] = 4
  #Уровень вывода ссобщений
params['silent'] = 1

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
#early_stopping_rounds - если через 100 кругов результат не будет улучшаться, выполнение остановится
#10000 - максимальный размер итераций
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=100, verbose_eval=10)

del d_train, d_valid

Обучение ...
[0]	train-mae:0.48808	valid-mae:0.481119
Multiple eval metrics have been passed: 'valid-mae' will be used for early stopping.

Will train until valid-mae hasn't improved in 100 rounds.
[10]	train-mae:0.402199	valid-mae:0.395444
[20]	train-mae:0.332673	valid-mae:0.326099
[30]	train-mae:0.276517	valid-mae:0.270132
[40]	train-mae:0.231317	valid-mae:0.225214
[50]	train-mae:0.195064	valid-mae:0.189327
[60]	train-mae:0.166123	valid-mae:0.160725
[70]	train-mae:0.143115	valid-mae:0.13805
[80]	train-mae:0.124972	valid-mae:0.120241
[90]	train-mae:0.110785	valid-mae:0.106352
[100]	train-mae:0.099816	valid-mae:0.095695
[110]	train-mae:0.091452	valid-mae:0.087611
[120]	train-mae:0.085137	valid-mae:0.081602
[130]	train-mae:0.080447	valid-mae:0.077224
[140]	train-mae:0.077012	valid-mae:0.074089
[150]	train-mae:0.074499	valid-mae:0.07184
[160]	train-mae:0.072682	valid-mae:0.070257
[170]	train-mae:0.071361	valid-mae:0.069146
[180]	train-mae:0.070406	valid-mae:0.068383
[190]	train-mae:0.069

In [9]:
print('Создание тестового набора данных ...')
#Объединение таблиц
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

del prop; gc.collect()

x_test = df_test[train_columns]
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)

del df_test, sample; gc.collect()

d_test = xgb.DMatrix(x_test)

del x_test; gc.collect()

Создание тестового набора данных ...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


2

In [10]:
print('Прогнозирование на тестовых данных ...')
#Предсказание на d_test
p_test = clf.predict(d_test)

del d_test; gc.collect()

sub = pd.read_csv('drive/MyDrive/zillow-prize-1/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

Прогнозирование на тестовых данных ...


In [11]:
print('Запись csv ...')
sub.to_csv('xgb_starter.csv', index=False, float_format='%.4f') # Thanks to @inversion

Запись csv ...
